In [ ]:
import inspect

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy

import scipy.optimize as spo

from scipy.interpolate import interp1d
from scipy.interpolate import InterpolatedUnivariateSpline

from scipy.integrate import solve_ivp

from types import SimpleNamespace
from copy import copy


def run_solve_ivp(system, slope_func, **options):
    """Computes a numerical solution to a differential equation.

    `system` must contain `init` with initial conditions,
    `t_end` with the end time.  Optionally, it can contain
    `t_0` with the start time.

    It should contain any other parameters required by the
    slope function.

    `options` can be any legal options of `scipy.integrate.solve_ivp`

    system: dictionary object
    slope_func: function that computes slopes

    returns: DataFrame
    """

    # make sure `system` contains `init`
    if 'init' not in system:
        msg = """It looks like the dictionary`system` does not contain `init`
                 as a key.  `init` should be a state pd.Series
                 object that specifies the initial condition:"""
        raise ValueError(msg)

    # make sure `system` contains `t_end`
    if 't_end' not in system:
        msg = """It looks like `system` does not contain `t_end`
                 as a key.  `t_end` should be the
                 final time:"""
        raise ValueError(msg)

    # the default value for t_0 is 0
    if 't_0' not in system:
        system['t_0'] = 0

    # try running the slope function with the initial conditions
    try:
        slope_func(system['t_0'], system['init'], system)
    except Exception as e:
        msg = """Before running scipy.integrate.solve_ivp, I tried
                 running the slope function you provided with the
                 initial conditions in `system` and `t=t_0` and I got
                 the following error:"""
        logger.error(msg)
        raise (e)

    # get the list of event functions
    events = options.get('events', [])

    # if there's only one event function, put it in a list
    try:
        iter(events)
    except TypeError:
        events = [events]

    for event_func in events:
        # make events terminal unless otherwise specified
        if not hasattr(event_func, 'terminal'):
            event_func.terminal = True

        # test the event function with the initial conditions
        try:
            event_func(system['t_0'], system['init'], system)
        except Exception as e:
            msg = """Before running scipy.integrate.solve_ivp, I tried
                     running the event function you provided with the
                     initial conditions in `system` and `t=t_0` and I got
                     the following error:"""
            logger.error(msg)
            raise (e)

    # get dense output (i.e. a continuous solution) unless otherwise specified
    if not 'dense_output' in options:
        options['dense_output'] = True

    # run the solver
    # a 'bunch' object is a dictionary whose values can be accessed
    # using attribute style syntax  (i.e. bunch1.t_0 rather than dict1['t_0'])
    bunch = solve_ivp(slope_func, [system['t_0'], system['t_end']], system['init'],
                      args=[system], **options)

    # 'bunch' contains numpy ndarrays called y and t.
    # y contains the results for state of the system over
    #  the whole simulation.  t contains the time
    # steps.  The following lines separate them into separate
    # arrays
    y = bunch.y
    t = bunch.t

    # get the column names from `init`
    columns = system['init'].keys()

    # evaluate the results at equally-spaced points
    # the first code block runs in t_eval is defined or if dense_output is false
    if 't_eval' in options or not options['dense_output']:
        results = pd.DataFrame(y.T, index=t,
                        columns=columns)
    else:
        # Assign the number of time steps for dense_output = True option
        num = 101
        # Define the solution at each of the time steps
        t_final = t[-1]
        t_array = linspace(system['t_0'], t_final, num)
        y_array = bunch.sol(t_array)
        # pack the results into a DataFrame
        results = pd.DataFrame(y_array.T, index=t_array,
                                   columns=columns)

    return results, bunch